In [1]:
import numpy as np
class project():
    def __init__(self):
        import pandas as pd
        import numpy as np
        self.pvto_data = pd.read_csv('pvto-data.csv')
        self.pvtw_data = pd.read_csv('pvtw-data.csv')
        self.pvtg_data = pd.read_csv('pvtg-data.csv')
        self.absperm = pd.read_csv('abs-perm.csv')
        self.rockfluid_data = pd.read_csv('rockproperties-data.csv')
        self.rockref_data = pd.read_csv('rockref-data.csv')
        self.rhostc_data = pd.read_csv('rhostc-data.csv')
        self.ngrid = pd.read_csv('n-grid.csv')
        self.dimension = pd.read_csv('res-dimension.csv')
        self.initialprop = pd.read_csv('top-initial_p_sw.csv')
        self.well_idx = pd.read_csv('well-indexes.csv')
        self.number_of_well = pd.read_csv('number-of-well.csv')
        self.qdata = pd.read_csv('q_data.csv')

        self.nwell = self.number_of_well['nwell'][0]
        #initial properties
        self.pi = self.initialprop['pi'][0]
        self.swi = self.initialprop['swi'][0]

        #reservoir gridblocks
        self.i_w = self.well_idx['i']
        self.j_w = self.well_idx['j']
        self.k_w = self.well_idx['k']
        self.nx = self.ngrid['nx'][0]
        self.ny = self.ngrid['ny'][0]
        self.nz = self.ngrid['nz'][0]
        self.dx = self.dimension['dx'][0]
        self.dy = self.dimension['dy'][0]
        self.dz = self.dimension['dz'][0]
        self.dltx = self.dx / self.nx
        self.dlty = self.dy / self.ny
        self.dltz = self.dz / self.nz
        self.vb = self.dltx * self.dlty * self.dltz
        self.kx = self.absperm['kx'][0]
        self.ky = self.absperm['ky'][0]
        self.kz = self.absperm['kz'][0]
        self.pgeox = 6.3283e-3*self.dlty*self.dltz*self.kx/self.dltx
        self.pgeoy = 6.3283e-3*self.dltx*self.dltz*self.ky/self.dlty
        self.pgeoz = 6.3283e-3*self.dltx*self.dlty*self.kz/self.dltz

        self.sw3d = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.swprev = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.p3d = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.pprev = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.pz = self.Pz(self.pi,self.nz,self.dltz)
        for i in range(0,self.nx):
            for j in range(0,self.ny):
                for k in range(0,self.nz):
                    self.sw3d[i][j][k] = self.swi
                    self.swprev[i][j][k] = self.swi
                    self.p3d[i][j][k] = self.pz[k]
                    self.pprev[i][j][k] = self.pz[k]

        #potential flow matrix
        self.ifo = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.igo = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.ibw = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.icw = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.idw = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.iew = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.ifw = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.igw = np.zeros((self.nx,self.ny,self.nz),dtype=float)

        #well() initialization
        self.qw = np.zeros(self.nwell,dtype=float)
        self.qo = np.zeros(self.nwell,dtype=float)
        self.dqwds = np.zeros(self.nwell,dtype=float)
        self.dqwdp = np.zeros(self.nwell,dtype=float)
        self.dqods = np.zeros(self.nwell,dtype=float)
        self.dqodp = np.zeros(self.nwell,dtype=float)

        #deriv() initialization
        self.J = np.zeros(4,dtype=float)
        self.Fo = 0
        self.Fw = 0

        #jacob() initialization
        self.Ja = np.zeros((self.nx,self.ny,self.nz,4),dtype=float)
        self.Jb = np.zeros((self.nx+1,self.ny,self.nz,4),dtype=float)
        self.Jc = np.zeros((self.nx+1,self.ny,self.nz,4),dtype=float)
        self.Jd = np.zeros((self.nx,self.ny+1,self.nz,4),dtype=float)
        self.Je = np.zeros((self.nx,self.ny+1,self.nz,4),dtype=float)
        self.Jf = np.zeros((self.nx,self.ny,self.nz+1,4),dtype=float)
        self.Jg = np.zeros((self.nx,self.ny,self.nz+1,4),dtype=float)

        self.fluxo = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.fluxw = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        
        self.acco = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.accw = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        
        self.r_fo = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.r_fw = np.zeros((self.nx,self.ny,self.nz),dtype=float)

        self.acc = np.zeros(4,dtype=float)

        self.t = 900
        self.dt = 1.0
        self.i_test = 0
        self.j_test = 0
        self.k_test = 0
        #solver
        self.jp = np.zeros((self.nx,self.ny,self.nz,7), dtype=int)
        self.jm = np.zeros((self.nx*self.ny*self.nz*2, 2*self.nx*self.ny*self.nz), dtype=float)
        self.jmm = np.zeros(2*self.nx*self.ny*self.nz, dtype=float)
    def lookup_oil(self,par,p): #par as parameter, p as pressure (psi)
        pvto_data = self.pvto_data
        i = 0
        if p < pvto_data['p'].iloc[i]:
            y = pvto_data[par].iloc[i]
        elif p > pvto_data['p'].iloc[-1]:
            y = pvto_data[par].iloc[-1]
        else:
            x1 = pvto_data['p'].iloc[i]
            x2 = pvto_data['p'].iloc[i+1]
            y1 = pvto_data[par].iloc[i]
            y2 = pvto_data[par].iloc[i+1]
            while p > pvto_data['p'].iloc[i+1]:
                i+=1
                x1 = pvto_data['p'].iloc[i]
                x2 = pvto_data['p'].iloc[i+1]
                y1 = pvto_data[par].iloc[i]
                y2 = pvto_data[par].iloc[i+1]
            y = y1 + ( (y2-y1)/(x2-x1) ) * (p-x1)
        return y
    def lookup_gas(self,par,p): #par as parameter, p as pressure (psi)
        pvtg_data = self.pvtg_data
        i = 0
        if p < pvtg_data['p'].iloc[i]:
            y = pvtg_data[par].iloc[i]
        elif p > pvtg_data['p'].iloc[-1]:
            y = pvtg_data[par].iloc[-1]
        else:
            x1 = pvtg_data['p'].iloc[i]
            x2 = pvtg_data['p'].iloc[i+1]
            y1 = pvtg_data[par].iloc[i]
            y2 = pvtg_data[par].iloc[i+1]
            while p > pvtg_data['p'].iloc[i+1]:
                i+=1
                x1 = pvtg_data['p'].iloc[i]
                x2 = pvtg_data['p'].iloc[i+1]
                y1 = pvtg_data[par].iloc[i]
                y2 = pvtg_data[par].iloc[i+1]
            y = y1 + ( (y2-y1)/(x2-x1) ) * (p-x1)
        return y
    def lookup_rockfluid(self,par,sw): #par as parameter, sw as saturation (fraction)
        rockfluid_data = self.rockfluid_data
        i = 0
        if sw < rockfluid_data['sw'].iloc[i]:
            y = rockfluid_data[par].iloc[i]
        elif sw > rockfluid_data['sw'].iloc[-1]:
            y = rockfluid_data[par].iloc[-1]
        else:
            x1 = rockfluid_data['sw'].iloc[i]
            x2 = rockfluid_data['sw'].iloc[i+1]
            y1 = rockfluid_data[par].iloc[i]
            y2 = rockfluid_data[par].iloc[i+1]
            while sw > rockfluid_data['sw'].iloc[i+1]:
                i+=1
                x1 = rockfluid_data['sw'].iloc[i]
                x2 = rockfluid_data['sw'].iloc[i+1]
                y1 = rockfluid_data[par].iloc[i]
                y2 = rockfluid_data[par].iloc[i+1]
            y = y1 + ( (y2-y1)/(x2-x1) ) * (sw-x1)
        return y
    def poten(self):
        import numpy as np
        nx = self.nx; ny = self.ny; nz = self.nz
        dltz = self.dltz
        p3d = self.p3d
        ifo = self.ifo; ifw = self.ifw; igo = self.igo; igw = self.igw; ibw = self.ibw; icw = self.icw; idw = self.idw; iew = self.iew
        for i in range(0,nx):
            for j in range(0,ny):
                for k in range(0,nz):
                    pa = p3d[i][j][k]
                    if k > 0: #to have neighbor grid: f, must have k index more than zero
                        pf = p3d[i][j][k-1]
                        pavg = 0.5 * (pa+pf)
                        pot_o = (pf - pa) + self.dno(pavg) * dltz
                        pot_w = (pf - pa) + self.dnw(pavg) * dltz
                        if pot_o > 0.0:
                            ifo[i][j][k] = 1
                        if pot_w > 0.0:
                            ifw[i][j][k] = 1
                    if k < nz-1: #to have neighbor grid: g, must have k index less than nz-1
                        pg = p3d[i][j][k+1]
                        pavg = 0.5 * (pa+pg)
                        pot_o = (pg - pa) - self.dno(pavg) * dltz
                        pot_w = (pg - pa) - self.dnw(pavg) * dltz
                        if pot_o > 0.0:
                            igo[i][j][k] = 1
                        if pot_w > 0.0:
                            igw[i][j][k] = 1
                    if i > 0: #to have neighbor grid: b, must have i index more than zero
                        pb = p3d[i-1][j][k]
                        pot_w = pb - pa
                        if pot_w > 0.0:
                            ibw[i][j][k] = 1
                    if i < nx-1: #to have neighbor grid: c, must have i index less than nx-1
                        pc = p3d[i+1][j][k]
                        pot_w = pc - pa
                        if pot_w > 0.0:
                            icw[i][j][k] = 1
                    if j > 0: #to have neighbor grid: d, must have j index more than zero
                        pd = p3d[i][j-1][k]
                        pot_w = pd - pa
                        if pot_w > 0.0:
                            idw[i][j][k] = 1
                    if j < ny-1: #to have neighbor grid: e, must have j index less than ny-1
                        pe = p3d[i][j+1][k]
                        pot_w = pe - pa
                        if pot_w > 0.0:
                            iew[i][j][k] = 1
    def well(self):
        import numpy as np
        qdata = self.qdata
        nw = self.nwell

        p3d = self.p3d
        sw3d = self.sw3d

        #time, in days, pandas format
        time = qdata['t']

        t = self.t

        #consist of 1.injection well; 2.production well
        i_w = self.i_w
        j_w = self.j_w
        k_w = self.k_w

        #call flow and its derivative parameters
        qw = self.qw
        qo = self.qo
        dqwds = self.dqwds
        dqwdp = self.dqwdp
        dqods = self.dqods
        dqodp = self.dqodp

        for n in range(0,nw): #n=0 : injector, n=1 : producer
            iw = i_w[n]
            jw = j_w[n]
            kw = k_w[n]

            p = p3d[iw][jw][kw]
            sw = sw3d[iw][jw][kw]

            idx = 0
            while t > time[idx]:
                idx+=1
            
            q = qdata.iloc[idx,[n+1]][0]
            qt = q * 5.6146

            if n == 0: #injector
                fw = 1
            else:
                M = (self.krw(sw)*self.muo(p)*self.bo(p))/(self.kro(sw)*self.muw(p)*self.bw(p))
                fw = M/(M+1)
            
            qw[n] = qt * fw
            qo[n] = qt * (1-fw)
            dqwds[n] = qw[n] * (1-fw) * (self.dkrw(sw)/self.krw(sw) - self.dkro(sw)/self.kro(sw))
            dqwdp[n] = qw[n] * (1-fw) * (self.dmuo(p)/self.muo(p)+self.dbo(p)/self.bo(p)-self.dmuw(p)/self.muw(p) +self.dbw(p)/self.bw(p))
            dqods[n] = -dqwds[n]
            dqodp[n] = -dqwdp[n]
    def deriv(self,swa,swn,pa,pn,dltz,ijkw,ijko,pgeo):
        """
        Return derivation of transmissibillity term as Jacobian coefficient as an J array
        J[0] = dFo/dSwn, J[1] = dFo/dpn, J[2] = dFw/dSwn, J[3] = dFw/dpn
        """
        if ijkw == 1:
            krwn = self.krw(swn)
            dkrwn = self.dkrw(swn)
        else:
            krwn = self.krw(swa)
            dkrwn = 0
        if ijko == 1:
            kron = self.kro(swn)
            dkron = self.dkro(swn)
        else:
            kron = self.kro(swa)
            dkron = 0
        pavg = (pa + pn) / 2
        muwbw_avg = (self.muw(pavg) * self.bw(pavg)) 
        muobo_avg = (self.muo(pavg) * self.bo(pavg)) 
        Tw = krwn / muwbw_avg * pgeo
        To = kron / muobo_avg * pgeo
        
        dTwswn = pgeo * dkrwn / muwbw_avg
        dToswn = pgeo * dkron / muobo_avg
        dTwpn = - pgeo * krwn / (2 * muwbw_avg) * (self.dmuw(pavg) / self.muw(pavg) + self.dbw(pavg) / self.bw(pavg))
        dTopn = - pgeo * kron / (2 * muobo_avg) * (self.dmuo(pavg) / self.muo(pavg) + self.dbo(pavg) / self.bo(pavg))
        
        dno = self.dno(pavg)
        dnw = self.dnw(pavg)
        ddno = self.ddno(pavg)
        ddnw = self.ddnw(pavg)
        
        J = self.J
        J[0] = dToswn * ( (pn-pa) - dno * dltz )
        J[1] = dTopn * ( (pn-pa) - dno * dltz )  + To * (1- ddno * dltz / 2)
        J[2] = dTwswn * ( (pn-pa) - dnw * dltz )
        J[3] = dTwpn * ( (pn-pa) - dnw * dltz )  + Tw * (1- ddnw * dltz / 2)
        self.Fo = To * ( (pn-pa) - dno * dltz )
        self.Fw = Tw * ( (pn-pa) - dnw * dltz )
    def inplace(self):
        import numpy as np
        nx = self.nx; ny = self.ny; nz = self.nz
        vb = self.vb
        P3d = self.p3d
        Sw3d = self.sw3d
        sum_oil = 0
        sum_gas = 0
        sum_water = 0
        for i in range(0,nx):
            for j in range(0,ny):
                for k in range(0,nz):
                    sum_oil+= vb * self.phi(P3d[i][j][k]) * (1-Sw3d[i][j][k]) / self.bo(P3d[i][j][k])
                    sum_gas+= vb * self.phi(P3d[i][j][k]) * self.rs(P3d[i][j][k]) * (1-Sw3d[i][j][k]) / self.bo(P3d[i][j][k])
                    sum_water+= vb * self.phi(P3d[i][j][k]) * Sw3d[i][j][k] / self.bw(P3d[i][j][k])
        print('OOIP (MMSTB) = ',round(sum_oil/5.6146/1e6,3))
        print('OGIP (BSCF) = ',round(sum_gas/1e9,3))
        print('OWIP (MMSTB) = ',round(sum_water/5.6146/1e6,3))
    def Pz(self,pi,nz,dltz):
        Pz = [pi]
        itr = 0
        while itr < nz-1:
            pa = Pz[itr]
            pb = pa + 50
            err = 50
            while err > 1e-4:
                pavg = (pa + pb)/2
                f_pb = pa + self.dno(pavg) * dltz - pb
                df_pb = self.ddno(pavg) * dltz / 2 - 1
                pbnew = pb - f_pb / df_pb
                err = abs(pbnew - pb)
                pb = pbnew
            Pz.append(pbnew)
            itr+=1
        return Pz
    def bo(self,p):
        return self.lookup_oil('bo',p)
    def rs(self,p):
        v_rs = self.lookup_oil('rs',p) * 1000 / 5.6146
        return v_rs
    def muo(self,p):
        return self.lookup_oil('muo',p)
    def bg(self,p):
        v_bg = self.lookup_gas('bg',p) * 5.6146 / 1000
        return v_bg
    def bw(self,p):
        pvtw_data = self.pvtw_data
        c = pvtw_data['cw'][0]
        x = c * (p - pvtw_data['pref'][0])
        Bw = pvtw_data['bw_ref'][0] / (1 + x + x**2/2)
        return Bw
    def muw(self,p):
        pvtw_data = self.pvtw_data
        cv = pvtw_data['viscosibility'][0]
        y = -cv * (p - pvtw_data['pref'][0])
        Muw = self.pvtw_data['muw_ref'][0] / (1 + y + y**2/2)
        return Muw
    def kro(self,sw):
        return self.lookup_rockfluid('kro',sw)
    def krw(self,sw):
        return self.lookup_rockfluid('krw',sw)
    def pcow(self,sw):
        return self.lookup_rockfluid('pcow',sw)  
    def rho_o(self,p):
        rhostc = self.rhostc_data
        rho_oil = (rhostc['rostc'][0] + self.rs(p) * rhostc['rgstc'][0]) / self.bo(p)
        return rho_oil
    def rho_g(self,p):
        rhostc = self.rhostc_data
        rho_gas = rhostc['rgstc'][0] / self.bg(p)
        return rho_gas
    def rho_w(self,p):
        rhostc = self.rhostc_data
        rho_water = rhostc['rwstc'][0] / self.bw(p)
        return rho_water
    def dno(self,p):
        dn_oil = self.rho_o(p) / 144
        return dn_oil
    def dnw(self,p):
        dn_water = self.rho_w(p) / 144
        return dn_water
    def dng(self,p):
        dn_gas = self.rho_g(p) / 144
        return dn_gas
    def phi(self,p):
        import math as m
        rock = self.rockref_data
        phi_rock = rock['phi0'][0] * m.exp( rock['cr'][0] * (p - rock['pref'][0]) )
        return phi_rock
    def dbo(self,p):
        p1 = p - 1e-3
        dBo = ( self.bo(p) - self.bo(p1) ) / (p - p1)
        return dBo
    def dmuo(self,p):
        p1 = p - 1e-3
        dMuo = ( self.muo(p) - self.muo(p1) ) / (p - p1)
        return dMuo
    def drs(self,p):
        p1 = p - 1e-3
        dRs = ( self.rs(p) - self.rs(p1) ) / (p - p1)
        return dRs
    def dbw(self,p):
        p1 = p - 1e-3
        dBw = ( self.bw(p) - self.bw(p1) )/ (p - p1)
        return dBw
    def dmuw(self,p):
        p1 = p - 1e-3
        dMuw = ( self.muw(p) - self.muw(p1) ) / (p - p1)
        return dMuw
    def drho_o(self,p):
        p1 = p - 1e-3
        drho_oil = ( self.rho_o(p) - self.rho_o(p1) ) / (p - p1)
        return drho_oil
    def drho_g(self,p):
        p1 = p - 1e-3
        drho_gas = ( self.rho_g(p) - self.rho_g(p1) ) / (p - p1)
        return drho_gas
    def drho_w(self,p):
        p1 = p - 1e-3
        drho_water = ( self.rho_w(p) - self.rho_w(p1) ) / (p - p1)
        return drho_water
    def ddno(self,p):
        p1 = p - 1e-3
        ddn_oil = ( self.dno(p) - self.dno(p1) ) / (p - p1)
        return ddn_oil
    def ddnw(self,p):
        p1 = p - 1e-3
        ddn_water = ( self.dnw(p) - self.dnw(p1) ) / (p - p1)
        return ddn_water
    def dphi(self,sw):
        sw1 = sw - 1e-3
        dPhi = ( self.phi(sw) - self.phi(sw1) ) / (sw - sw1)
        return dPhi
    def dkro(self,sw):
        sw1 = sw - 1e-3
        dKro = ( self.kro(sw) - self.kro(sw1) ) / (sw - sw1)
        return dKro
    def dkrw(self,sw):
        sw1 = sw - 1e-3
        dKrw = ( self.krw(sw) - self.krw(sw1) ) / (sw - sw1)
        return dKrw
    def dpcow(self,sw):
        sw1 = sw - 1e-3
        dPcow = ( self.pcow(sw) - self.pcow(sw1) ) / (sw - sw1)
        return dPcow
    def jacob(self):
        """
        Need to call well(t) before call this function
        """
        import numpy as np
        nx = self.nx
        ny = self.ny
        nz = self.nz
        nw = self.nwell
        sw3d = self.sw3d
        p3d = self.p3d
        vb = self.vb

        #consist of 1.injection well; 2.production well
        i_w = self.i_w
        j_w = self.j_w
        k_w = self.k_w
        #pgeo
        pgeox = self.pgeox
        pgeoy = self.pgeoy
        pgeoz = self.pgeoz

        i = self.i_test
        j = self.j_test
        k = self.k_test
        dt = self.dt
        swprev = self.swprev
        pprev = self.pprev

        #call initialization
        Ja = self.Ja
        Jb = self.Jb
        Jc = self.Jc
        Jd = self.Jd
        Je = self.Je
        Jf = self.Jf
        Jg = self.Jg

        fluxo = self.fluxo
        fluxw = self.fluxo
        
        acco = self.acco
        accw = self.accw
        
        r_fo = self.r_fo
        r_fw = self.r_fo

        acc = self.acc

        #well
        qo = self.qo
        qw = self.qw
        dqods = self.dqods
        dqodp = self.dqodp
        dqwds = self.dqwds
        dqwdp = self.dqwdp

        #potential
        ifo = self.ifo; ifw = self.ifw; igo = self.igo; igw = self.igw; ibw = self.ibw; icw = self.icw; idw = self.idw; iew = self.iew
        
        J = self.J
        for k in range(0,nz):
            for j in range(0,ny):
                for i in range(0,nx):
                    sa = sw3d[i][j][k]                
                    p = p3d[i][j][k]
                    
                    #B node
                    if i==0:
                        btxw = 0
                        btxo = 0
                    else:
                        pm1 = p3d[i-1][j][k]
                        sm1 = sw3d[i-1][j][k]
                        dltz = 0
                        self.deriv(sa,sm1,p,pm1,dltz,ibw[i][j][k],ibw[i][j][k],pgeox)
                        btxw = self.Fw
                        btxo = self.Fo            

                        for l in range(0,4):
                            Jb[i][j][k][l] = J[l]
                    #C node
                    if i==nx-1:
                        ctxw = 0
                        ctxo = 0
                    else:
                        pp1 = p3d[i+1][j][k]
                        sp1 = sw3d[i+1][j][k]
                        dltz = 0
                        self.deriv(sa,sp1,p,pp1,dltz,icw[i][j][k],icw[i][j][k],pgeox)
                        ctxw = self.Fw
                        ctxo = self.Fo

                        for l in range(0,4):
                            Jc[i][j][k][l] = J[l]
                    #D node
                    if j==0:
                        dtyw = 0
                        dtyo = 0
                    else:
                        pm1 = p3d[i][j-1][k]
                        sm1 = sw3d[i][j-1][k]
                        dltz = 0
                        self.deriv(sa,sm1,p,pm1,dltz,idw[i][j][k],idw[i][j][k],pgeoy)
                        dtyw = self.Fw
                        dtyo = self.Fo

                        for l in range(0,4):
                            Jd[i][j][k][l] = J[l]
                    #E node
                    if j==ny-1:
                        etyw = 0
                        etyo = 0
                    else:
                        pp1 = p3d[i][j+1][k]
                        sp1 = sw3d[i][j+1][k]
                        dltz = 0
                        self.deriv(sa,sp1,p,pp1,dltz,iew[i][j][k],iew[i][j][k],pgeoy)
                        etyw = self.Fw
                        etyo = self.Fo

                        for l in range(0,4):
                            Je[i][j][k][l] = J[l]
                
                    #F node
                    if k==0:
                        ftzw = 0
                        ftzo = 0
                    else:
                        pm1 = p3d[i][j][k-1]
                        sm1 = sw3d[i][j][k-1]
                        dltz = -self.dltz
                        self.deriv(sa,sm1,p,pm1,dltz,ifw[i][j][k],ifo[i][j][k],pgeoz)
                        ftzw = self.Fw
                        ftzo = self.Fo

                        for l in range(0,len(J)):
                            Jf[i][j][k][l] = J[l]
                    #G node
                    if k==nz-1:
                        gtzw = 0
                        gtzo = 0
                    else:
                        pp1 = p3d[i][j][k+1]
                        sp1 = sw3d[i][j][k+1]
                        dltz = self.dltz
                        self.deriv(sa,sp1,p,pp1,dltz,igw[i][j][k],igo[i][j][k],pgeoz)
                        gtzw = self.Fw
                        gtzo = self.Fo
                        
                        for l in range(0,len(J)):
                            Jg[i][j][k][l] = J[l]
                    
                    fluxw[i][j][k] = btxw + ctxw + dtyw + etyw + ftzw + gtzw
                    fluxo[i][j][k] = btxo + ctxo + dtyo + etyo + ftzo + gtzo

        for k in range(0,nz):
            for j in range(0,ny):
                for i in range(0,nx):
                    sa = sw3d[i][j][k]                
                    p = p3d[i][j][k]
                    pbef = pprev[i][j][k]
                    swbef = swprev[i][j][k]
                    phi = self.phi(p)
                    phin = self.phi(pbef)
                    dphi = self.dphi(p)
                    bw = self.bw(p)
                    bwn = self.bw(pbef)
                    dbw = self.dbw(p)
                    bo = self.bo(p)
                    bon = self.bo(pbef)
                    dbo = self.dbo(p)

                    accw = -vb/dt * (phi*sa/bw - phin*swbef/bwn)
                    acco = -vb/dt * (phi*(1-sa)/bo - phin*(1-swbef)/bon)

                    acc[0] = vb/dt * (phi/bo)
                    acc[1] = -vb/dt * (1-sa) * (dphi*bo-phi*dbo) / (bo**2)
                    acc[2] = -vb/dt * (phi/bw)
                    acc[3] = -vb/dt * sa * (dphi*bw-phi*dbw) / (bw**2)

                    ssw = 0
                    sso = 0
                    ss = np.zeros(4,dtype=float)
                    for l in range(0,nw):
                        if i == i_w[l] and j == j_w[l] and k == k_w[l]:
                            ssw = qw[l]
                            sso = qo[l]
                            ss[0] = dqods[l]
                            ss[1] = dqodp[l]
                            ss[2] = dqwds[l]
                            ss[3] = dqwdp[l]
                    #A node
                    for l in range(0,4):
                        Ja[i][j][k][l] = -Jb[i+1][j][k][l]-Jc[i-1][j][k][l]-Jd[i][j+1][k][l]-Je[i][j-1][k][l]-Jf[i][j][k+1][l]-Jg[i][j][k-1][l]+ acc[l] - ss[l]
                    r_fo[i][j][k] = fluxo[i][j][k] + acco - sso
                    r_fw[i][j][k] = fluxw[i][j][k] + accw - ssw
        print(fluxo[0][0][4])
        print(fluxw[0][0][4])
    def jm_positioner(self):
        jp = self.jp
        nx = self.nx
        ny = self.ny
        nz = self.nz
        # Absensi Jacobian
        count = 0
        for k in range(0, nz):
            for j in range(0, ny):
                for i in range(0, nx):
                    # Location A relative to i,j,k
                    jp[i][j][k][0] = count
                    count +=1
        # Neighbour Coordinator
        for k in range(0, nz):
            for j in range(0, ny):
                for i in range(0, nx):
                    # Location F relative of i,j,k
                    # Up
                    if(k!=0):
                        jp[i][j][k][1] = jp[i][j][k-1][0]
                    else:
                        jp[i][j][k][1] = -1
                    # Location D relative of i,j,k
                    # Back
                    if(j!=0):
                        jp[i][j][k][2] = jp[i][j-1][k][0]
                    else:
                        jp[i][j][k][2] = -1
                    # Location B relative of i,j,k
                    # Left
                    if(i!=0):
                        jp[i][j][k][3] = jp[i-1][j][k][0]
                    else:
                        jp[i][j][k][3] = -1
                    # Location C relative of i,j,k
                    # Right
                    if(i!=nx-1):
                        jp[i][j][k][4] = jp[i+1][j][k][0]
                    else:
                        jp[i][j][k][4] = -1
                    # Location E relative of i,j,k
                    # Front
                    if(j!=ny-1):
                        jp[i][j][k][5] = jp[i][j+1][k][0]
                    else:
                        jp[i][j][k][5] = -1
                    # Location G relative of i,j,k
                    # Down
                    if(k!=nz-1):
                        jp[i][j][k][6] = jp[i][j][k+1][0]
                    else:
                        jp[i][j][k][6] = -1
        return

    def jm_constructor(self):
        jm = self.jm
        jmm = self.jmm
        n = 0
        for k in range(0, nz):
            for j in range(0, ny):
                for i in range(0, nx):
                    # 2-Rows per grid
                    for h in range(0, 2):   # h={0,1}
                        # 7 Derivate Members
                        for m in range(0, 7):
                            # if(jp[i][j][k][m]!=-1):
                            for mm in range(0, 2):
                                if(m==0 and jp[i][j][k][m]!=-1):   # A
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 1111
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Ja[i][j][k][h * 2+mm]
                                elif(m==1 and jp[i][j][k][m]!=-1): # F
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 6666
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Jf[i][j][k][h * 2+mm]
                                elif(m==2 and jp[i][j][k][m]!=-1): # D
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 4444
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Jd[i][j][k][h * 2+mm]
                                elif(m==3 and jp[i][j][k][m]!=-1): # B
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 2222
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Jb[i][j][k][h * 2+mm]
                                elif(m==4 and jp[i][j][k][m]!=-1): # C
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 3333
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Jc[i][j][k][h * 2+mm]
                                elif(m==5 and jp[i][j][k][m]!=-1): # E
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 5555
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Je[i][j][k][h * 2+mm]
                                elif(m==6 and jp[i][j][k][m]!=-1):  # G
                                    # jm[n][jp[i][j][k][m] * 2 + mm] = 7777
                                    jm[n][jp[i][j][k][m] * 2 + mm] = self.Jg[i][j][k][h * 2+mm]
                        n+=1
        nrow = 0
        for k in range(0, nz):
            for j in range(0, ny):
                for i in range(0, nx):
                    jmm[nrow] = -self.Fo[i][j][k]
                    nrow+=1
                    jmm[nrow] = -self.Fw[i][j][k]
                    nrow+=1
        # if nnn == 1:
        #     with open("jcb.txt", "w+") as ww:
        #         for r in range(0, 2*Ngx*Ngy*Ngz):
        #             for c in range(0, 2*Ngx*Ngy*Ngz):
        #                 if(c!=2*Ngx*Ngy*Ngz-1):
        #                     ww.write(str(jm[r][c])+" ")
        #                 else:
        #                     ww.write(str(jm[r][c]))
        #                     ww.write("\n")
        return

In [2]:
# Main Program
from scipy.linalg import lu_factor, lu_solve
# Collecting Arrays
aTIME = []
aDT = []
aWATINJ = []
aOILPROD = []
aWATPROD = []
aWC = []
aWOR = []
aCUMINJ = []
aCUMOPROD = []
aCUMWPROD = []
aPWBINJ = []
aPWBPROD = []
aMB_ERR_OIL = []
aMB_ERR_WAT = []



print("Subprogram:Readdata/running")
f = project()
print("Subprogram:Readdata/success")
print("")

print("Subprogram:Initial_Cond/running")
print("Subprogram:Initial_Cond/success")
print("")

print("Subprogram:jm_positioner/running")
f.jm_positioner()
print("Subprogram:jm_positioner/success")
print("")

E_s = float(0.001)
E_p = float(0.1)
E_fo = float(1)
E_fw = float(5)

dSLIM = 0.02
dPLIM = 50

f.t = 0 #as time
f.dt = 2 #as dtime
tmax = 7600
cum_oilprod = 0
cum_watprod = 0
cum_watinj = 0
cum_oilinj = 0

nx = f.nx
ny = f.ny
nz = f.nz

P = self.pbef
S = self.swbef
p3d = self.p3d
sw3d = self.sw3d
while f.t<tmax:
    P = np.zeros((nx,ny,nz), dtype=float)
    S = np.zeros((nx,ny,nz), dtype=float)
    
    f.t = f.t + f.dt
    for k in range(0, nz):
        for j in range(0, ny):
            for i in range(0, nx):
                P[i][j][k]=p3d[i][j][k]
                S[i][j][k]=sw3d[i][j][k]
    
    print("Subprogram:Poten/running")
    f.poten()
    print("Subprogram:Poten/success")
    print("")

    c = 0
    niter = 0
    itermax = 100
    while c==0:
        niter += 1
        print("Subprogram:Well/running")
        f.well(f.t)
        print("Subprogram:Well/success")
        print("")

        print("Subprogram:Jacob/running")
        f.jacob()
        print("Subprogram:Jacob/success")
        print("")

        print("Subprogram:jm_creator/running")
        f.jm_constructor()
        print("Subprogram:jm_creator/success")
        print("")

        print("Subprogram:gauss/running")
        # sol = solve(jm, jmm)
        lu, piv = lu_factor(f.jm)
        sol = lu_solve((lu, piv), f.jmm)
        print("time: ", f.t)
        print("iter: ", niter)
        print("Subprogram:gauss/success")
        print("")

        # Update Values
        # Separate Solution to Sw & P
        x_dsw = np.zeros((nx,ny,nz), dtype=float)
        x_dp = np.zeros((nx,ny,nz), dtype=float)
        dr = 0
        for k in range(0, nz):
            for j in range(0, ny):
                for i in range(0, nx):
                    x_dsw[i][j][k] = sol[dr]
                    dr+=1
                    x_dp[i][j][k] = sol[dr]
                    dr+=1
        for k in range(0, nz):
            for j in range(0, ny):
                for i in range(0, nx):
                    sw3d[i][j][k] = sw3d[i][j][k]+x_dsw[i][j][k]
                    p3d[i][j][k] = p3d[i][j][k]+x_dp[i][j][k]
        x_dsw_max = np.amax(abs(x_dsw))
        x_dp_max = np.amax(abs(x_dp))
        fo_max = np.amax(abs(f.r_Fo))
        fw_max = np.amax(abs(f.r_Fw))

        # if(x_dp_max<E_p and x_dsw_max<E_s):
        #     c = 1
        if(fo_max<E_fo and fw_max<E_fw and x_dp_max<E_p and x_dsw_max<E_s):
            c = 1
        else:
            if(niter>itermax):
                # t = tmax
                f.dt = f.dt*0.5
                f.t=f.t-f.dt
            # if dt<10**-6:
            #     t = tmax
    
    print("Subprogram:Calc_Rem/running")
    calc_rem()
    print("Subprogram:Calc_Rem/success")
    print("")

    for i in range(0, Nw):
        if f.qw[i]>0:
            Qw = f.qw[i]
            Qo = f.qo[i]
            cum_watprod += Qw*f.dt
            # cum_watprod += Qw*dt/5.6146
            cum_oilprod += Qo*f.dt
            # cum_oilprod += Qo*dt/5.6146
        if f.qw[i]<0:
            Qi = abs(f.qw[i])
            cum_watinj += abs(f.qw[i])*dt
            # cum_watinj += abs(qw[i])*dt/5.6146
            cum_oilinj += abs(f.qo[i])*dt
            # cum_oilinj += abs(qo[i])*dt/5.6146

    mbew = (owip-rwip-cum_watprod+cum_watinj)/owip
    mbeo = (ooip-roip-cum_oilprod+cum_oilinj)/owip
    
    watcut = Qw/(Qo+Qw)
    wor = Qw/Qo

    aTIME.append(f.t)
    aDT.append(f.dt)
    aWATINJ.append(Qi)
    aOILPROD.append(Qo)
    aWATPROD.append(Qw)
    aWC.append(watcut)
    aWOR.append(wor)
    aCUMINJ.append(cum_watinj/1000)
    aCUMOPROD.append(cum_oilprod/1000)
    aCUMWPROD.append(cum_watprod/1000)
    aPWBINJ.append(p3d[0][0][4])
    aPWBPROD.append(p3d[4][4][4])
    aMB_ERR_OIL.append(mbeo)
    aMB_ERR_WAT.append(mbew)

    dPMAX = np.amax(abs(p3d-pbef))
    dSMAX = np.amax(abs(sw3d-swbef))

    dtold = f.dt
    dT_new_p = dPLIM/dPMAX
    dT_new_s = dSLIM/dSMAX
    f.dt = f.dt*min([dT_new_s, dT_new_p])
    if(f.dt/dtold>2):
        f.dt = dtold*2
    if(f.dt>30):
        f.dt = 30
    if(f.t<tmax and f.t+f.dt>tmax):
        f.dt = tmax - f.t

with open("resultsim_base.txt", "w+") as ww:
    ww.write("TIME DT WATINJ OILPROD WATPROD WC WOR CUMINJ CUMOPROD CUMWPROD PWBINJ PWBPROD MBEO MBEW")
    ww.write("\n")
    ww.write("Days Days STB/D STB/D STB/D % STB/STB STB STB STB psia psia dec. dec.")
    ww.write("\n")
    for x in range(0, len(aTIME)):
        ww.write(str(aTIME[x])+ " ")
        ww.write(str(aDT[x])+ " ")
        ww.write(str(aWATINJ[x])+ " ")
        ww.write(str(aOILPROD[x])+ " ")
        ww.write(str(aWATPROD[x])+ " ")
        ww.write(str(aWC[x])+ " ")
        ww.write(str(aWOR[x])+ " ")
        ww.write(str(aCUMINJ[x])+ " ")
        ww.write(str(aCUMOPROD[x])+ " ")
        ww.write(str(aCUMWPROD[x])+ " ")
        ww.write(str(aPWBINJ[x])+ " ")
        ww.write(str(aPWBPROD[x])+ " ")
        ww.write(str(aMB_ERR_OIL[x])+ " ")
        ww.write(str(aMB_ERR_WAT[x])+ " ")
        ww.write("\n")

print("Simulation Run Completed")

Subprogram:Readdata/running
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'rostc'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


TypeError: object of type 'NoneType' has no len()

In [36]:
a7.vb

17280000.0

In [38]:
a7 = project()

In [8]:
qdata = a7.qdata
qdata.iloc[1,[1]][0]

-1500

In [39]:
a7.t = 1000
i = 4; j = 4; k = 4

p3d = a7.p3d
sw3d = a7.sw3d
p3d[i][j][k] -= 100
sw3d[i][j][k] = 0.5

In [13]:
p3d[i][j][k]

2034.1349649947665

In [4]:
a7.well()

In [6]:
print('Qo = %f' %a7.qo[1]) 
print('Qw = %f' %a7.qw[1])
print ('dQw/dSw = %f' %a7.dqwds[1])
print ('dQw/dP = %f' %a7.dqwdp[1])
print ('dQo/dSw = %f' %a7.dqods[1])
print ('dQo/dP = %f' %a7.dqodp[1])

Qo = 1132.976701
Qw = 7288.923299
dQw/dSw = 16088.651480
dQw/dP = -0.067230
dQo/dSw = -16088.651480
dQo/dP = 0.067230


In [11]:
a7 = project()

In [12]:
#test deriv
nx = a7.nx
ny = a7.ny
nz = a7.nz
p3d = a7.p3d
sw3d = a7.sw3d
ibw = a7.ibw; icw = a7.icw; idw = a7.idw; iew = a7.iew; ifw = a7.ifw; ifo = a7.ifo; igw = a7.igw; igo = a7.igo;
for i in range(0,nx):
    for j in range(0,ny):
        for k in range(0,nz):
            if i%2 == 0:
                p3d[i][j][k] += 10
            if j%2 == 0:
                p3d[i][j][k] += 10
            if k%2 == 0:
                p3d[i][j][k] += 10
a7.poten()

In [14]:
i = 2; j = 1; k = 1
sw = sw3d[i][j][k] + 0.05
p = p3d[i][j][k]
swn = sw3d[i-1][j][k] + 0.05
pn = p3d[i-1][j][k]
dltz = 0
a7.deriv(sw,swn,p,pn,dltz,ibw[i][j][k],ibw[i][j][k],a7.pgeox)
print(a7.J[0],a7.J[1],a7.J[2],a7.J[3])
print(a7.Fw,a7.Fo)

-0.0 8.154846410035683 -0.0 0.0007630075528262261
-0.007630194938679939 -81.576212783534


In [15]:
swn_2 = sw3d[i][j-1][k] + 0.05
pn_2 = p3d[i][j-1][k]
dltz_2 = 0
a7.deriv(sw,swn_2,p,pn_2,dltz_2,idw[i][j][k],idw[i][j][k],a7.pgeoy)
print(a7.J[0],a7.J[1],a7.J[2],a7.J[3])
print(a7.Fw,a7.Fo)

-60.57147951079172 8.165976599576796 0.10649593528664136 0.0007630553177473256
0.007630433763287803 81.63186368529568


In [58]:
i = 2; j = 1; k = 1

In [59]:
swn_3 = sw3d[i][j][k+1] + 0.05
pn_3 = p3d[i][j][k+1]
dltz_3 = a7.dltz
a7.deriv(sw,swn_3,p,pn_3,dltz_3,igw[i][j][k],igw[i][j][k],a7.pgeoz)

In [60]:
print(a7.J[0],a7.J[1],a7.J[2],a7.J[3])
print(a7.Fw,a7.Fo)

-21291.065524376292 2869.0909551955815 25.42856136079843 0.2679563847613825
1.8219564215011954 28693.85678933149


In [76]:
pn,pn1

(2108.539447449435, 2108.539447449435)

In [23]:
a7 = project()

In [24]:
#tes jacob
nx = a7.nx
ny = a7.ny
nz = a7.nz

p3d = a7.p3d
sw3d = a7.sw3d
for i in range(0,nx):
    for j in range(0,ny):
        for k in range(0,nz):
            p3d[i][j][k] -= 20
            sw3d[i][j][k] += 0.01

In [25]:
a7.t = 800
a7.dt = 1
a7.i_test = 0
a7.j_test = 0
a7.k_test = 4

In [26]:
i = a7.i_test; j = a7.j_test; k = a7.k_test; t = a7.t
p3d[i][j][k] -= 20

In [27]:
a7.poten()
a7.well()
a7.jacob()

59314.597527662234
59314.597527662234


In [66]:
Jb = a7.Jb
Jc = a7.Jc
Jd = a7.Jd
Je = a7.Je
Jf = a7.Jf
Jg = a7.Jg
Ja = a7.Ja
r_Fw = a7.r_fw
r_Fo = a7.r_fo

for l in range(0,4):
    print(Jb[i][j][k][l]," ",Jc[i][j][k][l]," ",Jd[i][j][k][l])
    print(Je[i][j][k][l]," ",Jf[i][j][k][l]," ",Jg[i][j][k][l])
    print(Ja[i][j][k][l])
    print(" ")
print(r_Fw[i][j][k],r_Fo[i][j][k])

0.0   -120.98311893169051   0.0
-120.98311893169051   -42501.29441836821   0.0
1675537.5394264173
 
0.0   8.40006662844308   0.0
8.40006662844308   2948.3055897909026   0.0
-2830.5496220254245
 
0.0   0.21297893491578404   0.0
0.21297893491578404   86.78316650254504   0.0
-2339343.9407422896
 
0.0   0.00033704971363981496   0.0
0.00033704971363981496   0.11836158128661553   0.0
-3.1297580469766886
 
0.0 -14856.804957510887


In [37]:
a7.deriv(0.25,0.25,2100,2200,a7.dltz,1,1,a7.pgeox)

748.136252016869

In [40]:
t = a7.t
t = 1000
a7.t

900

In [11]:
import numpy
numpy.zeros(0).append(5)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [96]:
if t > max(time):
    idx = len(time)-1
elif t < min(time):
    idx = 0
else:
    while time
        idx+=1
time[idx]

2000

In [81]:
a7 = project()
t = a7.t
i = a7.i
j = a7.j
k = a7.k
p3d = a7.p3d
sw3d = a7.sw3d
p3d[i][j][k] = p3d[i][j][k]-100
sw3d[i][j][k] = 0.5
a7.well()

In [82]:
a7.i

0

In [73]:
i

4

In [58]:
t = 1000
i = 4; j = 4; k =4
p3d[i][j][k] = p3d[i][j][k]-100
sw3d[i][j][k] = 0.5
a7.well()

In [15]:
import numpy as np
np.zeros(4,dtype=float)

array([0., 0., 0., 0.])

In [20]:
a7.t

800

In [16]:
a7.dqodp

0

In [114]:
nx = a7.nx
ny = a7.ny
nz = a7.nz
p3d = a7.p3d
ibw = a7.ibw; icw = a7.icw; idw = a7.idw; iew = a7.iew; ifw = a7.ifw; ifo = a7.ifo; igw = a7.igw; igo = a7.igo;
for i in range(0,nx):
    for j in range(0,ny):
        for k in range(0,nz):
            if i%2 == 0:
                p3d[i][j][k] += 10
            if j%2 == 0:
                p3d[i][j][k] += 10
            if k%2 == 0:
                p3d[i][j][k] += 10
a7.poten()

In [41]:
a7.p3d

array([[[2130.        , 2128.53944745, 2147.07508575, 2145.60692243,
         2164.13496499],
        [2120.        , 2118.53944745, 2137.07508575, 2135.60692243,
         2154.13496499],
        [2130.        , 2128.53944745, 2147.07508575, 2145.60692243,
         2164.13496499],
        [2120.        , 2118.53944745, 2137.07508575, 2135.60692243,
         2154.13496499],
        [2130.        , 2128.53944745, 2147.07508575, 2145.60692243,
         2164.13496499]],

       [[2120.        , 2118.53944745, 2137.07508575, 2135.60692243,
         2154.13496499],
        [2110.        , 2108.53944745, 2127.07508575, 2125.60692243,
         2144.13496499],
        [2120.        , 2118.53944745, 2137.07508575, 2135.60692243,
         2154.13496499],
        [2110.        , 2108.53944745, 2127.07508575, 2125.60692243,
         2144.13496499],
        [2120.        , 2118.53944745, 2137.07508575, 2135.60692243,
         2154.13496499]],

       [[2130.        , 2128.53944745, 2147.07508575, 21

In [45]:
idw

array([[[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0]]])

In [115]:
n1=0; n2=0; n3=0; n4=0;
for i in range(0,nx):
    for j in range(0,ny):
        for k in range(0,nz):
            n1 += ibw[i][j][k] + icw[i][j][k] + idw[i][j][k] + iew[i][j][k]
            n2 += ifw[i][j][k] + igw[i][j][k] + ifo[i][j][k] + igo[i][j][k]
            n3 += ibw[i][j][k]*igw[i][j][k] + icw[i][j][k]*ifw[i][j][k]
            n4 += idw[i][j][k]*igo[i][j][k] + iew[i][j][k]*ifo[i][j][k]
print('n1 = %d' %n1)
print('n2 = %d' %n2)
print('n3 = %d' %n3)
print('n4 = %d' %n4)

n1 = 200
n2 = 200
n3 = 40
n4 = 40


In [116]:
#Since we have different reference index for matrix in FORTRAN and Python
#Then for given matrix calculation is called with corrected matrix in Python
#idx(i,j,k) 'fortran' = idx[i-1][j-1][k-1] 'python'

#ibw(1,2,3) = ibw[0][1][2]
print('ibw(1,2,3) = %d' %ibw[0][1][2])

#icw(2,3,4) = icw[1][2][3]
print('icw(2,3,4) = %d' %icw[1][2][3])

#idw(4,3,2) = idw[3][2][1]
print('idw(4,3,2) = %d' %idw[3][2][1])

#iew(3,2,1) = iew[2][1][0]
print('iew(3,2,1) = %d' %iew[2][1][0])

#ifw(5,1,5) = ifw[4][0][4]
print('ifw(5,1,5) = %d' %ifw[4][0][4])

#igw(4,2,4) = igw[3][1][3]
print('igw(4,2,4) = %d' %igw[3][1][3])

#ifw(3,3,3) = ifw[2][2][2]
print('ifw(3,3,3) = %d' %ifw[2][2][2])

#igw(2,2,2) = igw[1][1][1]
print('igw(2,2,2) = %d' %igw[1][1][1])

ibw(1,2,3) = 0
icw(2,3,4) = 1
idw(4,3,2) = 0
iew(3,2,1) = 1
ifw(5,1,5) = 0
igw(4,2,4) = 1
ifw(3,3,3) = 0
igw(2,2,2) = 1


In [10]:
a7 = project()
a7.jacob()

0.0 0.0 0.0 0.0 -0.0 0.0 1669731.2520968437

-62806.65775839178 23277.064198413696
0.0 8.466651604727138 0.0 8.466651604727138 2971.2523263026324 0.0 -2857.2812581441744

-62806.65775839178 23277.064198413696
0.0 0.0 0.0 0.0 -75.78563959309591 0.0 -2339917.6129918494

-62806.65775839178 23277.064198413696
0.0 0.00023057134629597518 0.0 0.00023057134629597518 0.08096367747192326 0.0 -2.9496793162394885

-62806.65775839178 23277.064198413696
0.0 0.0 0.0 0.0 0.0 -0.0 1674683.573452863

-12131.606663849276 23273.575797772603
8.446979125767331 0.0 8.446979125767331 0.0 0.0 2967.8111360336234 99.52689259042631

-12131.606663849276 23273.575797772603
0.0 0.0 0.0 0.0 0.0 -0.0 -2339428.047124008

-12131.606663849276 23273.575797772603
0.00023054671324371345 0.0 0.00023054671324371345 0.0 0.0 0.0809593528068615 -2.868573400188123

-12131.606663849276 23273.575797772603


In [5]:
for i in range(0,nx):
    for j in range(0,ny):
        for k in range(0,nz):
            if i%2 == 0:
                p3d[i][j][k] += 10
            if j%2 == 0:
                p3d[i][j][k] += 10
            if k%2 == 0:
                p3d[i][j][k] += 10

#call poten()
ifo = a6.poten('f','o')
igo = a6.poten('g','o')
ibw = a6.poten('b')
icw = a6.poten('c')
idw = a6.poten('d')
iew = a6.poten('e')
ifw = a6.poten('f')
igw = a6.poten('g')

#(i,j,k) = (5,5,5) as (4,4,4) in python
i=2; j=1; k=1
sa = sw3d[i][j][k] + 0.05
pa = p3d[i][j][k]

In [39]:
[0,0,0] == [9,0,0]

False

In [40]:
a7.p3d

array([[[2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499]],

       [[2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499],
        [2100.        , 2108.53944745, 2117.07508575, 2125.60692243,
         2134.13496499]],

       [[2100.        , 2108.53944745, 2117.07508575, 21

In [68]:
import numpy as np
[np.zeros((1)),np.zeros((1))] == [0,0]

True

In [6]:
sn1 = sw3d[i-1][j][k] + 0.05
pn1 = p3d[i-1][j][k]
dltz1 = 0

print ("J(1) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[0])
print ("J(2) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[1])
print ("J(3) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[2])
print ("J(4) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[3])
print ("btxw = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"tw"))
print ("btxo = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"to"))

J(1) = -0.000000
J(2) = 8.154846
J(3) = -0.000000
J(4) = 0.000763
btxw = -0.007630
btxo = -81.576213


In [7]:
sn2 = sw3d[i][j-1][k] + 0.05
pn2 = p3d[i][j-1][k]
dltz2 = 0

print ("J(1) = %f" %a6.deriv(sa,sn2,pa,pn2,dltz2,idw[i][j][k],idw[i][j][k],pgeoy,"J")[0])
print ("J(2) = %f" %a6.deriv(sa,sn2,pa,pn2,dltz2,idw[i][j][k],idw[i][j][k],pgeoy,"J")[1])
print ("J(3) = %f" %a6.deriv(sa,sn2,pa,pn2,dltz2,idw[i][j][k],idw[i][j][k],pgeoy,"J")[2])
print ("J(4) = %f" %a6.deriv(sa,sn2,pa,pn2,dltz2,idw[i][j][k],idw[i][j][k],pgeoy,"J")[3])
print ("dtyw = %f" %a6.deriv(sa,sn2,pa,pn2,dltz2,idw[i][j][k],idw[i][j][k],pgeoy,"tw"))
print ("dtyo = %f" %a6.deriv(sa,sn2,pa,pn2,dltz2,idw[i][j][k],idw[i][j][k],pgeoy,"to"))

J(1) = -60.571480
J(2) = 8.165977
J(3) = 0.106496
J(4) = 0.000763
dtyw = 0.007630
dtyo = 81.631864


In [8]:
sn3 = sw3d[i][j][k+1] + 0.05
pn3 = p3d[i][j][k+1]
dltz = a6.dz/nz

print ("J(1) = %f" %a6.deriv(sa,sn3,pa,pn3,dltz,igw[i][j][k],igo[i][j][k],pgeoz,"J")[0])
print ("J(2) = %f" %a6.deriv(sa,sn3,pa,pn3,dltz,igw[i][j][k],igo[i][j][k],pgeoz,"J")[1])
print ("J(3) = %f" %a6.deriv(sa,sn3,pa,pn3,dltz,igw[i][j][k],igo[i][j][k],pgeoz,"J")[2])
print ("J(4) = %f" %a6.deriv(sa,sn3,pa,pn3,dltz,igw[i][j][k],igo[i][j][k],pgeoz,"J")[3])
print ("gtzw = %f" %a6.deriv(sa,sn3,pa,pn3,dltz,igw[i][j][k],igo[i][j][k],pgeoz,"tw"))
print ("gtzo = %f" %a6.deriv(sa,sn3,pa,pn3,dltz,igw[i][j][k],igo[i][j][k],pgeoz,"to"))

J(1) = -21291.065524
J(2) = 2869.090955
J(3) = 25.428561
J(4) = 0.267956
gtzw = 1.821956
gtzo = 28693.856789


In [18]:
class project():
    def __init__(self):
        import pandas as pd
        import numpy as np
        self.pvto_data = pd.read_csv('pvto-data.csv')
        self.pvtw_data = pd.read_csv('pvtw-data.csv')
        self.pvtg_data = pd.read_csv('pvtg-data.csv')
        self.absperm = pd.read_csv('abs-perm.csv')
        self.rockfluid_data = pd.read_csv('rockproperties-data.csv')
        self.rockref_data = pd.read_csv('rockref-data.csv')
        self.rhostc_data = pd.read_csv('rhostc-data.csv')
        self.ngrid = pd.read_csv('n-grid.csv')
        self.dimension = pd.read_csv('res-dimension.csv')
        self.initialprop = pd.read_csv('top-initial_p_sw.csv')
        self.well_idx = pd.read_csv('well-indexes.csv')
        self.qprod = pd.read_csv('q_production.csv')
        self.qinj = pd.read_csv('q_injection.csv')

        #initial properties
        self.pi = self.initialprop['pi'][0]
        self.swi = self.initialprop['swi'][0]

        #reservoir gridblocks
        self.nx = self.ngrid['nx'][0]
        self.ny = self.ngrid['ny'][0]
        self.nz = self.ngrid['nz'][0]
        self.dx = self.dimension['dx'][0]
        self.dy = self.dimension['dy'][0]
        self.dz = self.dimension['dz'][0]
        self.dltx = self.dx / self.nx
        self.dlty = self.dy / self.ny
        self.dltz = self.dz / self.nz
        self.vb = self.dltx * self.dlty * self.dltz
        self.kx = self.absperm['kx'][0]
        self.ky = self.absperm['ky'][0]
        self.kz = self.absperm['kz'][0]
        self.pgeox = 6.3283e-3*self.dlty*self.dltz*self.kx/self.dltx
        self.pgeoy = 6.3283e-3*self.dltx*self.dltz*self.ky/self.dlty
        self.pgeoz = 6.3283e-3*self.dltx*self.dlty*self.kz/self.dltz

        self.sw3d = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.p3d = np.zeros((self.nx,self.ny,self.nz),dtype=float)
        self.pz = self.Pz(self.pi,self.nz,self.dltz)
        for i in range(0,self.nx):
            for j in range(0,self.ny):
                for k in range(0,self.nz):
                    self.sw3d[i][j][k] = self.swi
                    self.p3d[i][j][k] = self.pz[k]
    def lookup_oil(self,par,p): #par as parameter, p as pressure (psi)
        pvto_data = self.pvto_data
        if p in pvto_data['p'].values: #p is stated in pvto_data, need to call the value directly from dataframe
            return pvto_data[pvto_data['p']==p][par].values[0]
        else:
            p_0 = pvto_data[pvto_data['p'] < p]['p'].iloc[-1]
            p_1 = pvto_data[pvto_data['p'] > p]['p'].iloc[0]
            par_0 = pvto_data[pvto_data['p'] == p_0][par].values[0]
            par_1 = pvto_data[pvto_data['p'] == p_1][par].values[0]
            grad = ( par_1 - par_0 ) / ( p_1 - p_0 )
            par_value = grad * ( p - p_0 ) + par_0
            return par_value
    def lookup_gas(self,par,p): #par as parameter, p as pressure (psi)
        pvtg_data = self.pvtg_data
        if p in pvtg_data['p'].values: #p is stated in pvto_data, need to call the value directly from dataframe
            return pvtg_data[pvtg_data['p']==p][par].values[0]
        else:
            p_0 = pvtg_data[pvtg_data['p'] < p]['p'].iloc[-1]
            p_1 = pvtg_data[pvtg_data['p'] > p]['p'].iloc[0]
            par_0 = pvtg_data[pvtg_data['p'] == p_0][par].values[0]
            par_1 = pvtg_data[pvtg_data['p'] == p_1][par].values[0]
            grad = ( par_1 - par_0 ) / ( p_1 - p_0 )
            par_value = grad * ( p - p_0 ) + par_0
            return par_value
    def lookup_rockfluid(self,par,sw): #par as parameter, sw as water saturation (fraction)
        rockf_data = self.rockfluid_data
        if sw in rockf_data['sw'].values: #sw is stated in rockfluid_data, need to call the value directly from dataframe
            return rockf_data[rockf_data['sw']==sw][par].values[0]
        else:
            sw_0 = rockf_data[rockf_data['sw'] < sw]['sw'].iloc[-1]
            sw_1 = rockf_data[rockf_data['sw'] > sw]['sw'].iloc[0]
            par_0 = rockf_data[rockf_data['sw'] == sw_0][par].values[0]
            par_1 = rockf_data[rockf_data['sw'] == sw_1][par].values[0]
            grad = ( par_1 - par_0 ) / ( sw_1 - sw_0 )
            par_value = grad * ( sw - sw_0 ) + par_0
            return par_value
    def poten(self,idx,phase='w'):
        import numpy as np
        nx = self.nx; ny = self.ny; nz = self.nz
        dx = self.nx; dy = self.ny; dz = self.nz
        dltx = self.dltx; dlty = self.dlty; dltz = self.dltz
        
        #initialize matrix
        mat = np.zeros((nx,ny,nz),dtype=int)
        if phase == 'o' or phase == 'oil':
            if idx == 'f':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if k > 0: #to have index f, must have k index more than zero
                                pa = p3d[i][j][k]
                                pf = p3d[i][j][k-1]
                                pavg = 0.5 * (pa+pf)
                                pot_o = (pf - pa) + self.dno(pavg) * dltz
                                if pot_o > 0.0:
                                    mat[i][j][k] = 1 #ifo[i][j][k]
            elif idx == 'g':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if k < nz-1: #to have index f, must have k index less than nz-1
                                pa = p3d[i][j][k]
                                pg = p3d[i][j][k+1]
                                pavg = 0.5 * (pa+pg)
                                pot_o = (pg - pa) - self.dno(pavg) * dltz
                                if pot_o > 0.0:
                                    mat[i][j][k] = 1 #igo[i][j][k]
        elif phase == 'w' or phase == 'water':
            if idx == 'b':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if i > 0: #to have index b, must have i index more than zero
                                pa = p3d[i][j][k]
                                pb = p3d[i-1][j][k]
                                pot_w = pb - pa
                                if pot_w > 0.0:
                                    mat[i][j][k] = 1 #ibw[i][j][k]
            elif idx == 'c':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if i < nx-1: #to have index c, must have i index less than nx-1
                                pa = p3d[i][j][k]
                                pc = p3d[i+1][j][k]
                                pot_w = pc - pa
                                if pot_w > 0.0:
                                    mat[i][j][k] = 1 #icw[i][j][k]
            elif idx == 'd':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if j > 0: #to have index d, must have j index more than zero
                                pa = p3d[i][j][k]
                                pd = p3d[i][j-1][k]
                                pot_w = pd - pa
                                if pot_w > 0.0:
                                    mat[i][j][k] = 1 #idw[i][j][k]
            elif idx == 'e':
                for i in range(0,nx) :
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if j < ny-1: #to have index e, must have j index less than ny-1
                                pa = p3d[i][j][k]
                                pe = p3d[i][j+1][k]
                                pot_w = pe - pa
                                if pot_w > 0.0:
                                    mat[i][j][k] = 1 #iew[i][j][k]
            elif idx == 'f':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if k > 0: #to have index f, must have k index more than zero
                                pa = p3d[i][j][k]
                                pf = p3d[i][j][k-1]
                                pavg = 0.5 * (pa+pf)
                                pot_w = (pf - pa) + self.dnw(pavg) * dltz
                                if pot_w > 0.0:
                                    mat[i][j][k] = 1 #ifw[i][j][k]
            elif idx == 'g':
                for i in range(0,nx):
                    for j in range(0,ny):
                        for k in range(0,nz):
                            if k < nz-1: #to have index g, must have k index less than nz-1
                                pa = p3d[i][j][k]
                                pg = p3d[i][j][k+1]
                                pavg = 0.5 * (pa+pg)
                                pot_w = (pg - pa) - self.dnw(pavg) * dltz
                                if pot_w > 0.0:
                                    mat[i][j][k] = 1 #ifw[i][j][k]
        return mat
    def deriv(self,swa,swn,pa,pn,dltz,ijkw,ijko,pgeo,output):
        """
        Return derivation of transmissibillity term as Jacobian coefficient as an J array
        J[0] = dFo/dSwn, J[1] = dFo/dpn, J[2] = dFw/dSwn, J[3] = dFw/dpn
        """
        if ijkw == 1:
            krw = self.krw(swn)
            dkrwn = self.dkrw(swn)
        else:
            krw = self.krw(swa)
            dkrwn = 0
        if ijko == 1:
            kro = self.kro(swn)
            dkron = self.dkro(swn)
        else:
            kro = self.kro(swa)
            dkron = 0
        pavg = (pa + pn) / 2
        muwbw_avg = (self.muw(pavg) * self.bw(pavg)) 
        muobo_avg = (self.muo(pavg) * self.bo(pavg)) 
        Tw = krw / muwbw_avg * pgeo
        To = kro / muobo_avg * pgeo
        
        dTwswn = pgeo * dkrwn / muwbw_avg
        dToswn = pgeo * dkron / muobo_avg
        dTwpn = - pgeo * krw / (2 * muwbw_avg) * (self.dmuw(pavg) / self.muw(pavg) + self.dbw(pavg) / self.bw(pavg))
        dTopn = - pgeo * kro / (2 * muobo_avg) * (self.dmuo(pavg) / self.muo(pavg) + self.dbo(pavg) / self.bo(pavg))
        
        J = []
        J.append( dToswn * ((pn-pa) - self.dno(pavg) * dltz) ) #J[0]
        J.append( dTopn * ( (pn-pa) - self.dno(pavg) *dltz )  + To * (1- self.ddno(pavg) * dltz / 2) ) #J[1]
        J.append( dTwswn * ((pn-pa) - self.dnw(pavg) * dltz) ) #J[2]
        J.append( dTwpn * ( (pn-pa) - self.dnw(pavg) *dltz )  + Tw * (1- self.ddnw(pavg) * dltz / 2) ) #J[3]
        
        Fo = To * ( (pn-pa) - self.dno(pavg) * dltz )
        Fw = Tw * ( (pn-pa) - self.dnw(pavg) * dltz )
        if output == 'tw' or output == 'Tw':
            return Fw
        elif output =='to' or output == 'To':
            return Fo
        elif output =='J':
            return J
    def Pz(self,pi,nz,dltz):
        Pz = [pi]
        itr = 0
        while itr < nz-1:
            pa = Pz[itr]
            pb = pa + 50
            err = 50
            while err > 1e-4:
                pavg = (pa + pb)/2
                f_pb = pa + self.dno(pavg) * dltz - pb
                df_pb = self.ddno(pavg) * dltz / 2 - 1
                pbnew = pb - f_pb / df_pb
                err = abs(pbnew - pb)
                pb = pbnew
            Pz.append(pbnew)
            itr+=1
        return Pz
    def inplace(self):
        import numpy as np
        pi = self.pi; swi = self.swi
        nx = self.nx; ny = self.ny; nz = self.nz
        dx = self.nx; dy = self.ny; dz = self.nz
        dltx = self.dltx; dlty = self.dlty; dltz = self.dltz
        vb = self.vb
        P3d = self.p3d
        Sw3d = self.sw3d
        sum_oil = 0
        sum_gas = 0
        sum_water = 0
        for i in range(0,nx):
            for j in range(0,ny):
                for k in range(0,nz):
                    sum_oil+= vb * self.phi(P3d[i][j][k]) * (1-Sw3d[i][j][k]) / self.bo(P3d[i][j][k])
                    sum_gas+= vb * self.phi(P3d[i][j][k]) * self.rs(P3d[i][j][k]) * (1-Sw3d[i][j][k]) / self.bo(P3d[i][j][k])
                    sum_water+= vb * self.phi(P3d[i][j][k]) * Sw3d[i][j][k] / self.bw(P3d[i][j][k])
        print('OOIP (MMSTB) = ',round(sum_oil/5.6146/1e6,3))
        print('OGIP (BSCF) = ',round(sum_gas/1e9,3))
        print('OWIP (MMSTB) = ',round(sum_water/5.6146/1e6,3))
    def well(self,t,i,j,k):
        #well identification
        i_idx = self.well_idx['i']
        j_idx = self.well_idx['j']
        k_idx = self.well_idx['k']
        #rate and rate derivative calculation
        if [i,j,k] == [i_idx[1],j_idx[1],k_idx[1]]: #producer well
            #according to index of P3d and Sw3d start from zero then
            i=i-1; j=j-1; k=k-1
            fw = 1 / (1 + 1 / ( self.krw(Sw3d[i][j][k]) * self.muo(P3d[i][j][k]) / self.muw(P3d[i][j][k]) / self.kro(Sw3d[i][j][k]) ) )
            qtot = self.qprd(t) * 5.6146
            qw = qtot * fw
            qo = qtot * (1-fw)
            dQwdSw = qw * (1-fw) * ( self.dkrw(Sw3d[i][j][k])/self.krw(Sw3d[i][j][k]) - self.dkro(Sw3d[i][j][k])/self.kro(Sw3d[i][j][k]))
            dQwdP = qw  * (1-fw) * ( self.dmuo(P3d[i][j][k])/self.muo(P3d[i][j][k]) + self.dbo(P3d[i][j][k])/self.bo(P3d[i][j][k]) - self.dmuw(P3d[i][j][k])/self.muw(P3d[i][j][k]) - self.dbw(P3d[i][j][k])/self.bw(P3d[i][j][k]))
            dQodSw = -dQwdSw
            dQodP = -dQwdP
            print ('Producer Well')
            print ('Qw (ft3/d) = %d' %qw)
            print ('Qo (ft3/d) = %d' %qo)
            print ('dQw/dSw = %d' %dQwdSw)
            print ('dQw/dP = %d' %dQwdP)
            print ('dQo/dSw = %d' %dQodSw)
            print ('dQo/dP = %d' %dQodP)
        elif [i,j,k] == [i_idx[0],j_idx[0],k_idx[0]]: #injector well
            i=i-1; j=j-1; k=k-1
            fw = 1 #water cut
            qtot = self.qin(t) * 5.6146
            qw = qtot * fw
            qo = qtot * (1-fw)
            dQwdSw = qw * (1-fw) * ( self.dkrw(Sw3d[i][j][k])/self.krw(Sw3d[i][j][k]) - self.dkro(Sw3d[i][j][k])/self.kro(Sw3d[i][j][k]))
            dQwdP = qw  * (1-fw) * ( self.dmuo(P3d[i][j][k])/self.muo(P3d[i][j][k]) + self.dbo(P3d[i][j][k])/self.bo(P3d[i][j][k]) - self.dmuw(P3d[i][j][k])/self.muw(P3d[i][j][k]) - self.dbw(P3d[i][j][k])/self.bw(P3d[i][j][k]))
            dQodSw = -dQwdSw
            dQodP = -dQwdP
            print ('Injector Well')
            print ('Qw (ft3/d) = %d' %qw)
            print ('Qo (ft3/d) = %d' %qo)
            print ('dQw/dSw = %d' %dQwdSw)
            print ('dQw/dP = %d' %dQwdP)
            print ('dQo/dSw = %d' %dQodSw)
            print ('dQo/dP = %d' %dQodP)
        else:
            print ('Not a well, try configure gridblock (i,j,k) for well index')
    def qin(self,t):
        qinj = self.qinj
        if t > max(qinj['t']):
            return qinj['q'].values[-1]
        else:
            return qinj[qinj['t']>=t]['q'].values[0]
    def qprd(self,t):
        qprod = self.qprod
        if t > max(qprod['t']):
            return qprod['q'].values[-1]
        else:
            return qprod[qprod['t']>=t]['q'].values[0]
    def bo(self,p):
        return self.lookup_oil('bo',p)
    def rs(self,p):
        v_rs = self.lookup_oil('rs',p) * 1000 / 5.6146
        return v_rs
    def muo(self,p):
        return self.lookup_oil('muo',p)
    def bg(self,p):
        v_bg = self.lookup_gas('bg',p) * 5.6146 / 1000
        return v_bg
    def bw(self,p):
        c = self.pvtw_data['cw'].values[0]
        x = c * (p - self.pvtw_data['pref'].values[0])
        Bw = self.pvtw_data['bw_ref'].values[0] / (1 + x + x**2/2)
        return Bw
    def muw(self,p):
        cv = self.pvtw_data['viscosibility'].values[0]
        y = -cv * (p - self.pvtw_data['pref'].values[0])
        Muw = self.pvtw_data['muw_ref'].values[0] / (1 + y + y**2/2)
        return Muw
    def kro(self,sw):
        return self.lookup_rockfluid('kro',sw)
    def krw(self,sw):
        return self.lookup_rockfluid('krw',sw)
    def pcow(self,sw):
        return self.lookup_rockfluid('pcow',sw)  
    def rho_o(self,p):
        rhostc = self.rhostc_data
        rho_oil = (rhostc['rostc'].values[0] + self.rs(p) * rhostc['rgstc'].values[0]) / self.bo(p)
        return rho_oil
    def rho_g(self,p):
        rhostc = self.rhostc_data
        rho_gas = rhostc['rgstc'].values[0] / self.bg(p)
        return rho_gas
    def rho_w(self,p):
        rhostc = self.rhostc_data
        rho_water = rhostc['rwstc'].values[0] / self.bw(p)
        return rho_water
    def dno(self,p):
        dn_oil = self.rho_o(p) / 144
        return dn_oil
    def dnw(self,p):
        dn_water = self.rho_w(p) / 144
        return dn_water
    def dng(self,p):
        dn_gas = self.rho_g(p) / 144
        return dn_gas
    def phi(self,p):
        import math as m
        rock = self.rockref_data
        phi_rock = rock['phi0'].values[0] * m.exp( rock['cr'].values[0] * (p - rock['pref'].values[0]) )
        return phi_rock
    def dbo(self,p):
        p1 = p - 1e-3
        dBo = ( self.bo(p) - self.bo(p1) ) / (p - p1)
        return dBo
    def dmuo(self,p):
        p1 = p - 1e-3
        dMuo = ( self.muo(p) - self.muo(p1) ) / (p - p1)
        return dMuo
    def drs(self,p):
        p1 = p - 1e-3
        dRs = ( self.rs(p) - self.rs(p1) ) / (p - p1)
        return dRs
    def dbw(self,p):
        p1 = p - 1e-3
        dBw = ( self.bw(p) - self.bw(p1) )/ (p - p1)
        return dBw
    def dmuw(self,p):
        p1 = p - 1e-3
        dMuw = ( self.muw(p) - self.muw(p1) ) / (p - p1)
        return dMuw
    def drho_o(self,p):
        p1 = p - 1e-3
        drho_oil = ( self.rho_o(p) - self.rho_o(p1) ) / (p - p1)
        return drho_oil
    def drho_g(self,p):
        p1 = p - 1e-3
        drho_gas = ( self.rho_g(p) - self.rho_g(p1) ) / (p - p1)
        return drho_gas
    def drho_w(self,p):
        p1 = p - 1e-3
        drho_water = ( self.rho_w(p) - self.rho_w(p1) ) / (p - p1)
        return drho_water
    def ddno(self,p):
        p1 = p - 1e-3
        ddn_oil = ( self.dno(p) - self.dno(p1) ) / (p - p1)
        return ddn_oil
    def ddnw(self,p):
        p1 = p - 1e-3
        ddn_water = ( self.dnw(p) - self.dnw(p1) ) / (p - p1)
        return ddn_water
    def dphi(self,sw):
        sw1 = sw - 1e-3
        dPhi = ( self.phi(sw) - self.phi(sw1) ) / (sw - sw1)
        return dPhi
    def dkro(self,sw):
        sw1 = sw - 1e-3
        dKro = ( self.kro(sw) - self.kro(sw1) ) / (sw - sw1)
        return dKro
    def dkrw(self,sw):
        sw1 = sw - 1e-3
        dKrw = ( self.krw(sw) - self.krw(sw1) ) / (sw - sw1)
        return dKrw
    def dpcow(self,sw):
        sw1 = sw - 1e-3
        dPcow = ( self.pcow(sw) - self.pcow(sw1) ) / (sw - sw1)
        return dPcow

In [19]:
a6 = project()
sw3d = a6.sw3d
p3d = a6.p3d
nx = a6.nx; ny = a6.ny; nz = a6.nz
pgeox = a6.pgeox; pgeoy = a6.pgeoy; pgeoz = a6.pgeoz

for i in range(0,nx):
    for j in range(0,ny):
        for k in range(0,nz):
            if i%2 == 0:
                p3d[i][j][k] += 10
            if j%2 == 0:
                p3d[i][j][k] += 10
            if k%2 == 0:
                p3d[i][j][k] += 10

In [20]:
ifo = a6.poten('f','o')
igo = a6.poten('g','o')
ibw = a6.poten('b')
icw = a6.poten('c')
idw = a6.poten('d')
iew = a6.poten('e')
ifw = a6.poten('f')
igw = a6.poten('g')

In [21]:
i=2; j=1; k=1
sa = sw3d[i][j][k] + 0.05
pa = p3d[i][j][k]

In [22]:
sn1 = sw3d[i-1][j][k] + 0.05
pn1 = p3d[i-1][j][k]
dltz1 = 0

print ("J(1) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[0])
print ("J(2) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[1])
print ("J(3) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[2])
print ("J(4) = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"J")[3])
print ("btxw = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"tw"))
print ("btxo = %f" %a6.deriv(sa,sn1,pa,pn1,dltz1,ibw[i][j][k],ibw[i][j][k],pgeox,"to"))

J(1) = -0.000000
J(2) = 8.154846
J(3) = -0.000000
J(4) = 0.000763
btxw = -0.007630
btxo = -81.576213


In [57]:
a6.dno(pa/2+pn1/2)

0.3161226629080029

In [58]:
a7.dno(p/2+pn/2)

0.39239972636730197

In [64]:
pa/2+pn1/2

2113.539447449435

In [63]:
p/2+pn/2

2118.539447449435

In [65]:
pa,p

(2118.539447449435, 2128.539447449435)

In [16]:
sw3d[i][j][k]

1.450000000000001